In [1]:
library(SnapATAC)
library(Seurat)
library(Signac)
library(genomation)
library(GenomicRanges)
library(parallel)
library(foreach)

Loading required package: Matrix

Loading required package: rhdf5

Warning message:
“no DISPLAY variable so Tk is not available”
Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.3.2 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects ar

Specify file path

In [2]:
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/PBMC/1.prepare_data/2.merge_peaks.240624/pairs.rds"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/PBMC/1.prepare_data/2.merge_peaks.240624/matrix.atac.rds"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/PBMC/1.prepare_data/2.merge_peaks.240624/matrix.rna.rds"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/PBMC/2.Genome_wide_prediction/SnapATAC/PBMC.240624/"

In [3]:
n.cores = 16

Import candidate E-G pairs

In [4]:
pairs.E2G = readRDS(path.pairs.E2G)

Import ATAC matrix

In [5]:
matrix.atac_count = readRDS(path.matrix.atac_count)
matrix.atac = BinarizeCounts(matrix.atac_count)
rm(matrix.atac_count)

Import RNA matrix

In [6]:
matrix.rna_count = readRDS(path.matrix.rna_count)
matrix.rna_count = matrix.rna_count[,colnames(matrix.atac)]
matrix.rna_count = matrix.rna_count[rowSums(matrix.rna_count) > 0,]
matrix.rna = NormalizeData(matrix.rna_count)
rm(matrix.rna_count)

In [7]:
pairs.E2G.filter = pairs.E2G[pairs.E2G$TargetGene %in% rownames(matrix.rna) &
                             pairs.E2G$PeakName %in% rownames(matrix.atac)]

Run SnapATAC prediction

In [8]:
PredictionSnapATAC = function(pairs.E2G,
                              data.RNA,
                              data.ATAC,
                              n.core = n.cores){
  my.cluster <- parallel::makeCluster(
    n.core,
    type = "PSOCK"
  )
  doParallel::registerDoParallel(cl = my.cluster)
  
  pairs.E2G =  
    foreach (index.tmp = 1:length(pairs.E2G),
             .combine = 'c',
             .packages = c("Matrix",
                           "GenomicRanges")) %dopar% {

                             pairs.E2G.tmp = pairs.E2G[index.tmp]
                             
                             model.tmp = summary(stats::glm(data.ATAC[pairs.E2G.tmp$PeakName,] ~ data.RNA[pairs.E2G.tmp$TargetGene,], 
                                                            family = binomial(link = "logit")))[["coefficients"]]
                             mcols(pairs.E2G.tmp)[,c("beta","sd","z","p")] = 
                               t(data.frame(model.tmp[2,]))
                             
                             pairs.E2G.tmp
                           }
  
  parallel::stopCluster(cl = my.cluster)
  
  return(pairs.E2G)
}

In [9]:
dir.create(paste(dir.output,"chr",sep = "/"),recursive = T)

In [10]:
chr.done = dir(paste(dir.output,"chr",sep = "/"))
chr.run = as.character(unique(seqnames(pairs.E2G.filter)))
chr.run = chr.run[!chr.run %in% chr.done]

In [11]:
for(chr.tmp in chr.run){
  print(chr.tmp)
  pairs.E2G.chr.res = pairs.E2G.filter[seqnames(pairs.E2G.filter) == chr.tmp]
  mcols(pairs.E2G.chr.res)[,c("beta","sd","z","p")] = NA
  pairs.E2G.chr.res = PredictionSnapATAC(pairs.E2G.chr.res,
                                         matrix.rna[rownames(matrix.rna) %in% pairs.E2G.chr.res$TargetGene,],
                                         matrix.atac[rownames(matrix.atac) %in% pairs.E2G.chr.res$PeakName,],
                                         n.cores)
  saveRDS(pairs.E2G.chr.res,
          paste(dir.output,"chr",chr.tmp,sep = "/"))
  rm (pairs.E2G.chr.res)
}

[1] "chr1"
[1] "chr10"
[1] "chr11"
[1] "chr12"
[1] "chr13"
[1] "chr14"
[1] "chr15"
[1] "chr16"
[1] "chr17"
[1] "chr18"
[1] "chr19"
[1] "chr2"
[1] "chr20"
[1] "chr21"
[1] "chr22"
[1] "chr3"
[1] "chr4"
[1] "chr5"
[1] "chr6"
[1] "chr7"
[1] "chr8"
[1] "chr9"
[1] "chrX"


Merge results

In [12]:
chr.merge = as.character(unique(seqnames(pairs.E2G.filter)))
list.res = list()
for(chr.tmp in chr.merge){
    list.res[[chr.tmp]] = readRDS(paste(dir.output,"chr",chr.tmp,sep = "/"))
}
pairs.E2G.res = unlist(as(list.res, "GRangesList"))

Save results

In [13]:
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

GRanges object with 5424230 ranges and 7 metadata columns:
       seqnames              ranges strand |  TargetGene               PeakName
          <Rle>           <IRanges>  <Rle> | <character>            <character>
  chr1     chr1 100028331-100029533      * |         AGL chr1-100028331-10002..
  chr1     chr1 100028331-100029533      * |      CDC14A chr1-100028331-10002..
  chr1     chr1 100028331-100029533      * |         DBT chr1-100028331-10002..
  chr1     chr1 100028331-100029533      * |        DPH5 chr1-100028331-10002..
  chr1     chr1 100028331-100029533      * |       EXTL2 chr1-100028331-10002..
   ...      ...                 ...    ... .         ...                    ...
  chrX     chrX     9995651-9996434      * |        WWC3   chrX-9995651-9996434
  chrX     chrX     9997538-9997892      * |       CLCN4   chrX-9997538-9997892
  chrX     chrX     9997538-9997892      * |      GPR143   chrX-9997538-9997892
  chrX     chrX     9997538-9997892      * |       TBL1X   ch

In [14]:
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "beta",
                         "sd",
                         "z",
                         "p")]
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,TargetGene,CellType,beta,sd,z,p
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,100028331,100029533,AGL,K562,0.074926264,0.08922900,0.83970753,4.010724e-01
chr1,100028331,100029533,CDC14A,K562,-0.108273037,0.03480878,-3.11050945,1.867649e-03
chr1,100028331,100029533,DBT,K562,0.029630647,0.11797883,0.25115223,8.016964e-01
chr1,100028331,100029533,DPH5,K562,0.192646446,0.09348261,2.06077310,3.932469e-02
chr1,100028331,100029533,EXTL2,K562,0.418282866,0.21097645,1.98260450,4.741162e-02
chr1,100028331,100029533,FRRS1,K562,-0.086316817,0.30132815,-0.28645454,7.745300e-01
chr1,100028331,100029533,GPR88,K562,-8.357268559,165.04144960,-0.05063739,9.596145e-01
chr1,100028331,100029533,MFSD14A,K562,-0.532356656,0.39842302,-1.33615939,1.814972e-01
chr1,100028331,100029533,RTCA,K562,0.162072681,0.08303560,1.95184575,5.095652e-02


In [15]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.9 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] foreach_1.5.2        GenomicRanges_1.54.1 GenomeInfoDb_1.38.1 
 [4] IRanges_2.36.0       S4Vectors_0.40.2     BiocGene